In [1]:
!pip install opendatasets --quiet
import opendatasets as od
od.download('https://www.kaggle.com/datasets/fadyelkbeer/arabic-text-summarization-30-000')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  omaratef3221


Your Kaggle Key:

  ········


100%|██████████| 30.3M/30.3M [00:01<00:00, 20.8MB/s]


In [39]:
import pandas as pd
import random
import re 
import nltk
nltk.download('punkt')


data_df = pd.read_csv("arabic-text-summarization-30-000/wikiHow.csv")


[nltk_data] Downloading package punkt to /Users/omaratef/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Original: 
 الإسعافات الأولية السليمة هامة وضرورية للمساعدة في منع انتشار العدوى البكتيرية، لكن لا يجب عليك محاولة علاج جرح شديد على سطح الجلد بنفسك. إذا كان الجرح عميق أو واسع أو ينزف الكثير من الدماء، عليك طلب المساعدة الطبية في الحال. إذا عالجت جرح بيدين متسخة، ستزيد من احتمالية الإصابة بعدوى بكتيرية. اغسل يديك بمياه دافئة وصابون مضاد للبكتيريا لمدة 20 ثانية ونشفهم جيدا. ارتد قفازات الفينيل أو اللاتكس نظيفة إذا كانت متوفرة.  تجنب قفزات اللاتكس إن كنت تعاني من حساسية ضد مادة اللاتكس. أما لو كان النزيف شديدا، فاطلب العناية الطبية على الفور. لا تحاول علاج الجروح الشديدة بنفسك. اذهب إلى غرفة طوارئ أو اتصل بالنجدة. أبق الجرح تحت تدفق خفيف من الماء لتنظيفه. لا تستخدم الصابون على الجرح إلا لو كان متسخا بشكل واضح. نظف حول الجرح برفق باستخدام صابون خفيف إن لم يكن الجرح متسخا. لا تستخدم ماء الأكسجين (بيروكسيد الهيدروجين) أيضا في تنظيف الجرح. يمكن أن يعرقل ماء الأكسجين (بيروكسيد الهيدروجين) عملية شفاء الجرح.  لو لاحظت أية شظايا في الجرح، حاول إزالتها بملقط مطهر بالكحول. إذا كنت لا تشعر بالراحة

In [40]:
counter = 0
def delete_links(input_text):
    pettern  = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    out_text = re.sub(pettern, ' ', input_text)
    return out_text

def delete_repeated_characters(input_text):
    pattern  = r'(.)\1{2,}'
    out_text = re.sub(pattern, r"\1\1", input_text)
    return out_text

def remove_extra_spaces(input_text):
    replace = ' +'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

def replace_letters(input_text):
    replace = {"أ": "ا","ة": "ه","إ": "ا","آ": "ا","": ""}
    replace = dict((re.escape(k), v) for k, v in replace.items()) 
    pattern = re.compile("|".join(replace.keys()))
    out_text = pattern.sub(lambda m: replace[re.escape(m.group(0))], input_text)
    return out_text

def clean_text(input_text):
    replace = r'[^\u0621-\u064A\u0660-\u0669\u06F0-\u06F90-9]'
    out_text = re.sub(replace, " ", input_text)
    words = nltk.word_tokenize(out_text)
    #words = [word for word in words if word.isalpha()]
    out_text = ' '.join(words)
    return out_text

def remove_vowelization(input_text):
    vowelization = re.compile(""" ّ|َ|ً|ُ|ٌ|ِ|ٍ|ْ|ـ""", re.VERBOSE)
    out_text = re.sub(vowelization, '', input_text)
    return out_text

def delete_stopwords(input_text):
    stop_words = set(nltk.corpus.stopwords.words("arabic") + nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    wnl = nltk.WordNetLemmatizer()
    lemmatizedTokens =[wnl.lemmatize(t) for t in tokens]
    out_text = [w for w in lemmatizedTokens if not w in stop_words]
    out_text = ' '.join(out_text)
    return out_text

def stem_text(input_text):
    st = ISRIStemmer()
    tokenizer = nltk.tokenize.WhitespaceTokenizer()
    tokens = tokenizer.tokenize(input_text)
    out_text = [st.stem(w) for w in tokens]
    out_text = ' '.join(out_text)
    return out_text


def text_prepare(input_text, ar_text):
    global counter
    counter +=1

    #out_text = delete_links(input_text)
    #out_text = delete_repeated_characters(out_text)
    #out_text = delete_stopwords(input_text)
    out_text = clean_text(input_text)
    #out_text = remove_extra_spaces(out_text)
    if(counter%100==0):
        print(counter,'\n',out_text)
    return out_text

In [41]:
data_df['text'] = data_df['text'].apply(text_prepare, args=(True,))
data_df['summary'] = data_df['summary'].apply(text_prepare, args=(True,))

100 
 لا تنتظر حتى ليلة الامتحان النهائي وتبدأ المذاكرة وخاصة في مادة مثل الأدب الإنجليزي سيطلب منك الإجابة على أسئلة تحليلية وأخرى تخص المحتوى لذلك يجب أن تبدأ المذاكرة مبكرا لكي تصبح على دراية تامة بالأجزاء الصعبة في مادتك ضع في اعتبارك أنك ليس عليك فقط تلخيص موضوع القصة أو معرفة أسماء بعض الشخصيات تحتاج إلى معرفة ما هو أبعد من ذلك عن نصك دون كل التفاصيل التي تتذكرها من مذاكرتك الأولى لنصوصك بالإضافة إلى أي شيء تتذكره من المحاضرات لا تخدع نفسك من خلال اختلاس النظر إلى ملاحظاتك أو نصك دون فقط ما تتذكره بالفعل وتثق في ذلك يكون ذلك بمثابة القاعدة الأساسية والتي ستكشف أي ثغرات في معرفتك يجب أن تكون على دراية ببعض المصطلحات الرئيسية قبل دخولك لأي اختبار أو امتحان في الأدب الإنجليزي ومن أمثلة تلك المصطلحات المقطع الشعري والمفارقة والجناس والمتكلم واللغة التصويرية فهم بعض هذه المصطلحات الرئيسية سيساعدك كثيرا في النجاح على الرغم من أنه لا يتوقع منك أن تلم بكافة المصطلحات الأدبية هناك العديد من الإرشادات المتاحة التي تساعدك في إيجاد تعريفات للمفاهيم الأدبية الهامة وإليك فيما يلي بعض المصطلحات

1000 
 إذا جربت كل تلك الطرق ولا زلت غير قادر على الذهاب إلى الحمام قد يتكون لديك انسداد في الأمعاء إذا استمر الإمساك لأسابيع يجب عليك الفحص لاستبعاد مشاكل طبية أكثر خطورة يجب عليك مراجعة الطبيب فورا إذا كان لديك أعراض مثل التشنج أو النوبات أو الدوخة أو التعب يمكنك سؤال طبيبك عن موعد الارتجاع البيولوجي هذه استشارة خاصة ستعلمك كيفية الاسترخاء وتشديد عضلات منطقة الحوض تحدث إلى طبيبك إذا كنت تتناول أي أدوية يوجد احتمال أن تكون تلك الأدوية هي السبب في الإمساك يمكن أن يساعدك مساج البطن إذا كنت تعاني من مشاكل إمساك على المدى الطويل يستغرق المساج حوالي 10 إلى 20 دقيقة ويمكنك القيام به أثناء وقوفك أو جلوسك أو استلقائك يمكن لهذه المساجات التقليل من الحاجة إلى الاستخدام المنتظم للملينات كما تقوم بتخفيف انتفاخ البطن لا ينصح بتدليك البطن للجميع لذا تحدث إلى طبيبك أولا لا ينبغي للمرأة الحامل أن تقوم بمساج ولا ينبغي لشخص له تاريخ مرضي مع انسداد الأمعاء الخبيثة يمكنك التحدث مع طبيبك بشأن الأدوية المتاحة لك لعلاج الإمساك تعمل هذه الأدوية على سحب المياه إلى الأمعاء يسرع ذلك من حركة الأمعاء سيقوم الطبيب

2000 
 أغلب الذين يخافون من القطط على معرفة كاملة بأنها غير مؤذية وجميلة ولكن مع ذلك يشعرون بالخوف كنتيجة لرد الفعل من المخ وهو أمر لا يمكن التحكم به الفوبيا عبارة عن سلوك مكتسب قد يمر الشخص بتجربة سيئة مع قطة ما فيبدأ عقله دون أن يشعر بربط جميع الأفكار السلبية بالقطط مثل المرض أو قد يكون تعلم الخوف من القطط عن طريق مشاهدة ذعر والديه في وجودهم أثناء طفولته تشترك العديد من الأجزاء في المخ في جعلك تشعر بالفوبيا من القطط ولهذا قد تستغرق بعض الوقت لإعادة تدريب مخك بتغيير رد فعله حول القطط عند تحديدك لهذه الأفكار المحبطة يمكنك أن تبدأ في تقييمها ستلاحظ في النهاية أن جميع هذه الأفكار تتكون من واحد أو أكثر من هذه التشوهات المعرفية التخمين هو أن يفترض الشخص النتائج لموقف ما دون وجود دليل مادي يدعمه على سبيل المثال قد تفكر في هذه القطة ستقوم بخدشي حتى ولو لم يكن لديك أي اتصال مع قطط من قبل التعميم وهو أن يحدث موقف واحد ويقوم الشخص بتعميمه على جميع المواقف الأخرى على سبيل المثال قد تعتقد أن قطة صديقي خدشتني من عامين ولهذا فإن جميع القطط عنيفة التهويل حيث تتوقع نتيجة سلبية على وشك الحدوث وتعتقد أ

2500 
 تظهر أيقونة التطبيق بلون أسود وتحتوي على نوتة موسيقية بيضاء سيؤدي ذلك إلى فتح شاشة البحث اضغط بعد ذلك على زر البحث في لوحة المفاتيح جرب استيراد جهات اتصالك أو أصدقاء فيسبوك إن لم تكن ترغب بالبحث عن شخص معين انقر على لسان تبويب مستخدمين حال النقر بصورة عرضية على أحد ألسنة التبويب الأخرى أصوات أو وسوم مثلا سيؤدي ذلك إلى تغير لون زر متابعة من الأخضر إلى الرمادي وظهور كلمة متابع
2600 
 حتى تستخدم زر الطاقة في حاسوبك في إيقاف تشغيله فإنك بحاجة إلى تغيير الإعدادت للوصول إلى الإعدادات تحتاج إلى فتح لوحة التحكم لفتح لوحة التحكم إما أن تبحث عنها في شاشة بأن تكتب أو تضغط على مفتاح وتحدد لوحة التحكم من القائمة التي تظهر لك يمكنك الوصول إليها عن طريق كتابة في حقل البحث في أو يمكنك أن تجدها في قسم من يفتح لك ذلك صفحة جديدة يمكنك تخصيص ما يتم فعله عندما تضغط على زر الطاقة يدويا في حاسوبك انقر فوق القائمة المنسدلة بجوار حدد من قائمة الخيارات سيبدأ حاسوبك تلقائيا في إيقاف التشغيل عند الضغط على زر الطاقة في الحاسوب وهذه الطريقة هي إحدى الطرق الآمنة لإيقاف التشغيل حيث أنها تتم كما لو حددت من قائم

3500 
 إن كان شريكك يؤخر ارتباط عاطفي أو جسدي بك فمن المحتمل أن يكون في محاولة مستمرة لتحديد مشاعره نحوك أعط شريكك مساحة للشك كن صبورا واترك له وقت لأخذ القرار والتعرف عليك أكثر يسري هذا على الحالتين فقد يكون شريكك غير واثق من مشاعرك نحوه لأن أي شخص يتطلع لمتابعة علاقة عاطفية سيود الحصول على بعض التأمينات من حيث كون المشاعر متبادلة لذا كن صريحا مع شريكك بشأن نواياك إن كنت واثقا من مشاعرك أخبر شريكك أنك كنت تفكر مؤخرا بجدية في علاقتكما وأنك مستعد لممارسة الجنس وعلى الرغم من ذلك سيكون من الأفضل أن تنتظر الحصول على بعض الإشارات الإيجابية قبل اقتراح ممارسة الجنس في هذه النقطة إن كانت خبرتك عن الجنس هي ما شاهدته في الأفلام فقط فسيكون هذا ما تتوقع الحصول عليه يمكن القول بأن توقعاتك لا تتماشى فعليا مع الواقع فالخيال لا ينتمي لأرض الواقع إن لم تعي أن الجنس هو قرار هام وله عواقب جدية قد تحتاج للانتظار قليلا بعد بغض النظر عن خبرتك أنت أو شريكك فالجنس قد يكون فوضاويا وغريبا ستكون مثيرة مرتكم الأولى ولكن لا تتوقع الحصول على شيء يشبه الأفلام السينمائية توقعاتك غير الواقعية غالبا ما ستبين أنك لم تمع

...

29100 
 ارفع كلا الذراعين بشكل مستقيم على الجانب من أول الذراع وصولا إلى كف اليد متأكدا من أنهما موازيين لمستوى الأرض بعد ذلك اثن أحد الذراعين للداخل اثن ذراعا واحدا فقط الأيمن أو الأيسر حسب رغبتك مع الإبقاء على الذراع الآخر ممتدا كما هو دون ثني
29200 
 يجب أن تحصل على نفس التجربة التي سيحصل عليها أي ضيف عادي هذا لأن المطاعم تعاملك معاملة خاصة قد تغير تقييمك إذا علموا أنك ناقد طعام بدلا من إخبارهم أنك هناك لتقييم طعاهم حاول أن تدخل المطعم وتجلس على طاولة عادية وتعامل كزبون طبيعي تقترح رابطة صحفي نقد الطعام تجنب أي مناسبات كبرى كالافتتاحات أو الحفلات إلخ حتى لا تخاطر باقتراب أحد الطهاه منك سعيا وراء تقييم جيد إذا كنت ناقد طعام شهير يجب أن تحجز طاولة باسم مستعار يجب أن تجلب معك أيضا مفكرة صغيرة أو مسجل صغير يمكنك تدوين ملاحظاتك عليه أو استخدم هاتفك لفعل ذلك لكتابة تقييم رائع ومتوازن يجب أن تسجل ملاحظاتك هل احتجت إلى حجز طاولة وما المدة التي تطلبها تحضير الطاولة أين المطعم وكيف تصف الجوار كيف كان مكان انتظار السيارات يمكنك أن تخصص جزء صغيرا من مقالك لتلك التفاصيل لكن ضعها في عين الاعتبار 

34900 
 اعرف أكثر عن المنطقة التي تقطن بها قم بتخزين الماء والطعام داخل المنزل قم بجمع الإمدادات اللازمة قم بإعداد صندوق الطوارئ لوضعه بالسيارة قم بوضع خطة للطوارئ قم بتنصيب مولد للكهرباء احرص على تشذيب الأشجار والشجيرات حول منزلك اشترك في خطة تأمين مناسبة
35000 
 استخدم الركام كحشو لملء الفراغات استخدم عربة يد ثقيلة استأجر سلة نفايات من إحدى شركات التخلص منها اتصل بشركات دفن النفايات واسأل عن تكلفة التخلص منها خذ الخرسانة إلى مكان دفن النفايات أعد استخدام الخرسانة المتكسرة في أعمال منزلية أخرى
35100 
 اطلع على السلسلة الثانية من الأرقام المطبوعة في أدنى شيك مصرفي في حال توفر شيك تحت يديك اطلع على نسخة من كشف حسابك البنكي ورقية أو إلكترونية في حال توفر لك الوصول لها استخدم خدمات البنك عبر الهاتف المحمول أو تطبيقات الهاتف الذكي اتصل بخدمة عملاء البنك إذا فشلت كل محاولاتك الأخرى
35200 
 استخدم المسطرة أو شريط القياس بوضعها عند قاعدة القضيب المنتصب اجعل قضيبك مستويا على المسطرة أو شريط القياس حدد قياساتك
35300 
 اسق البقلة أبعد الحيوانات الأليفة اقطف الزهور الميتة خلص نباتك من الأعشاب الض

41300 
 اشطف فمك بماء ملحي دافىء استخدم كرة قطنية لوضع زيت القرنفل اذهب إلى طبيب الأسنان إذا كانت مشكلة الأسنان التي تعاني منها قوية اعلم مخاطر استخدام زيت القرنفل
41400 
 حددي ما تظنينه سبب عجزك عن الثقة بخطيبك لا تتعجلي وضع استنتاجات تأملي علاقاتك السابقة تحدثي إلى خطيبك عن شعورك احرصي على إلزام نفسك بالأمر ذاته ابذلي جهدا لتقوية علاقتك بخطيبك تعرفي على علامات مشاكل الثقة الأكثر خطورة فكري مليا في المنبع الآخر لمشاكل الثقة هذه الجئي لمختص مؤهل في الصحة النفسية
41500 
 انقلي عجينة الخبز إلى المجمد دون تغطيتها لمدة ساعة إلى ساعتين اكتبي التاريخ على الغلاف البلاستيكي حتى تعرفي متى تستخدمين العجينة أعيدي العجينة إلى المجمد واتركيها فترة تصل إلى 6 أشهر اتركي الثلج يذوب عن العجينة لأربع ساعات على الأقل في درجة حرارة الغرفة أعدي عجينة الخبز وفقا لتعليمات الوصفة
41600 
 تعرفي على دورك في الموقف ضعي حدودا شخصية دافعي عن حقيقتك دون الشعور بالذنب اتركي الرجل إذا رفض احترامك
41700 
 قل لها أنا أحبك أظهر لها حبك اكتب لها رسالة تصرح لها فيها بحبك كن مراعيا لرد فعلها
41800 
 تفقدي قدميك رطبي قدميك 

48200 
 استبعد احتمالية تعلق المشكلة بمرشح الوقود احصل على مقياس ضغط وقود وصل مقياس الضغط بأداة اختبار مضخة الوقود اطلب من صديق تشغيل المحرك أثناء تفقدك للمقياس
48300 
 احفظ نسخة احتياطية من نظامك قبل المتابعة أعد تهيئة القرص الصلب باستخدام أسطوانة التثبيت اختر خيار إعادة تهيئة القرص الصلب عندما يطلب منك ذلك اتبع كل الإرشادات المتعلقة بعملية إعادة تهيئة القرص الصلب
48400 
 استخدم بخاخ محلول الملح متى شعرت بأنف محتقن ضع في اعتبارك استخدام محقن لمبة للأطفال والرضع اسحب المخاط الموجود في أنف الطفل بالمحقن اللمبة حافظ على قواعد الصحة العامة بعد استخدام المحقن اللمبة كرر هذه العملية مرتين إلى ثلاث مرات في اليوم حافظ على نسبة السوائل في جسمك كن لطيفا وأنت تتخلص من المخاط اذهب للطبيب ليفحص الطفل أو الرضيع
48500 
 قم بتطوير طقوس تمشيط قطتك اقض وقتا هادئا مع قطتك وفر لقطتك أماكن مختلفة للنوم وفر لقطتك العديد من أعمدة الخدش أعط قطتك الكثير من الألعاب كي تلعب بها أضف المزيد من الإبداع لوقت اللعب مع قطتك قم بخلق تجربة بصرية مثيرة لقطتك قدم لقطتك الطعام الموسمي الشهي
48600 
 افتح سفاري انتقل إلى ال

54600 
 ثبت برنامج إن ماب افتح سطر الأوامر افحص منافذ الجهاز المستهدف قم بإجراء فحص معدل أخرج نتائج الفحص إلى ملف إكس إم إل
54700 
 استخدم طلقات الطيور لرياضة الرماية للعبة صيد كبيرة يمكن استخدام الرصاص احصل على الخراطيش ذات الحجم المناسب لبندقيتك فكر في الحصول على بعض الأعيرة النارية لاحظ حجم طلقة الذخيرة
54800 
 ابدأ في تجهيز الملعب ارم الكرة اضرب الكرة واركض أنه المباراة
54900 
 اعتمدي على الثدي الأصغر حجما في الرضاعة استعملي مضخة الثدي على الثدي الأصغر حجما أثناء الرضاعة بدلي بين الثديين في الرضاعة إلى أن يتماثلان في الحجم اجعلي طفلك يرضع من الثدي الأقل تفضيلا
55000 
 ضع سلة مهملات قريبة ارتد قفازات وقناع استخرج المواد الصلبة استخرج كتل البول استبدل أي فضلات مفقودة
55100 
 تعامل مع باقي الظفر أوقف أي نزيف نظف الإصابة تماما ضع مرهم مضاد حيوي ضع غطاء على الإصبع غير الضمادة يوميا
55200 
 فهم وظيفة الإفرازات المهبلية التعرف على شكل الإفرازات المهبلية الطبيعية تعرفي على السبب البديهي للتغير في الإفرازات لا تقلقي بشأن تنظيف الإفرازات المهبلية
55300 
 مارس النشاطات ابذل مجهودا ذهنيا شاقا 

In [47]:
index = random.randint(0, data_df.shape[0])


print("Data sample: ", index)

print("Original: \n", data_df["text"].iloc[index])
print("="*30)
print("Summarized: \n", data_df["summary"].iloc[index])

Data sample:  15041
Original: 
 استحم بشكل يومي وبحد أقصى كل ثلاثة أيام فمن الضروري أن تستحم بصورة دورية حتى مع عدم تعرقك بشكل كبير أو مع عدم تعرقك على الإطلاق في اليوم الثالث اغسل أسنانك بالفرشاة مرتين يوميا ونظفها بخيط التنظيف مرة واحدة يوميا لا يهتم الكثير من الأشخاص بالاعتناء بالأسنان وهو الأمر الذي يندمون عليه لاحقا عند إصابتهم بالتسوس والحاجة للذهاب إلى طبيب الأسنان تذكر أن الابتسامة الساحرة تصنع الفرق استعمل مزيل التعرق يوميا لا أحد يرغب في ملاحظة رائحة جسدك الكريهة وهو الأمر الذي يمكنك منعه بسهولة من خلال استعمال مزيل التعرق الواجب الاحتفاظ به بشكل دوري كي تنقذ نفسك وملابسك من الرائحة قد يفيدك الظهور بشكل وضيع في أثناء عمر المراهقة المبكرة ولكن هذا لن يجديك نفعا الآن ولهذا احرص على حلاقة الشارب المريب الهيئة عند نموه في وجهك انزع الشعر ما بين الحاجبين أو قم بحلاقته افعل هذا في جميع الأحوال اغسل وجهك يوميا يساعدك ذلك في التخلص من حب الشباب والرؤوس السوداء ولكن تذكر أن حب الشباب طبيعي في مرحلة المراهقة أما إذا كنت تعاني من الرؤوس السوداء فيمكنك الاستعانة بالمستحضرات التي تحتوي عل